### Llama 7B

In [ ]:
class PromptTemplate:
    system_prompt = None
    user_messages = []
    model_replies = []

    def __init__(self, system_prompt=None):
        self.system_prompt = system_prompt

    def add_user_message(self, message: str, return_prompt=True):
        self.user_messages.append(message)
        if return_prompt:
            return self.build_prompt()

    def add_model_reply(self, reply: str, includes_history=True, return_reply=True):
        reply_ = reply.replace(self.build_prompt(), "") if includes_history else reply
        self.model_replies.append(reply_)
        if len(self.user_messages) != len(self.model_replies):
            raise ValueError(
                "Number of user messages does not equal number of system replies."
            )
        if return_reply:
            return reply_

    def get_user_messages(self, strip=True):
        return [x.strip() for x in self.user_messages] if strip else self.user_messages

    def get_model_replies(self, strip=True):
        return [x.strip() for x in self.model_replies] if strip else self.model_replies

    def clear_chat_history(self):
        self.user_messages.clear()
        self.model_replies.clear()

    def build_prompt(self):
        if self.user_messages == [] and self.model_replies == []:
            return f"<s>[INST] <<SYS>>\n{self.system_prompt}\n<</SYS>> [/INST]</s>"
        
        elif len(self.user_messages) != len(self.model_replies) + 1:
            raise ValueError(
                "Error: Expected len(user_messages) = len(model_replies) + 1. Add a new user message!"
            )

        if self.system_prompt is not None:
            SYS = f"[INST] <<SYS>>\n{self.system_prompt}\n<</SYS>>"
        else:
            SYS = ""

        CONVO = ""
        SYS = "<s>" + SYS
        for i in range(len(self.user_messages) - 1):
            user_message, model_reply = self.user_messages[i], self.model_replies[i]
            conversation_ = f"{user_message} [/INST] {model_reply} </s>"
            if i != 0:
                conversation_ = "[INST] " + conversation_
            CONVO += conversation_

        CONVO += f"[INST] {self.user_messages[-1]} [/INST]"

        return SYS + CONVO

In [3]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

def setup_llama(model_directory:str="../models/llama/model" , tokenizer_directory:str="../models/llama/tokenizer'", hg_access_token:str=""):
    """
    Function to download llama2 7b model

    Args:
        model_directory (str): directory to the path with llama model, defaults to ../../models/llama/model'
        tokenizer_directory (str): directory to the path with llama tokenizer, defaults to ../../models/llama/tokenizer'
        hg_access_token (str): hugging face access token

    Returns:
        0 for success and 1 for failure
    """
    if len(hg_access_token) <=0:
        print("Not a valid access token")
        return 1

    if not os.path.isdir(model_directory):
        os.makedirs(model_directory) 
        print("Llama Model directory not found, directory created")

    if not os.path.isdir(tokenizer_directory):
        os.makedirs(tokenizer_directory) 
        print("Llama tokenizer directory not found, directory created")

    print("Preparing to download LLama")

    try:
        llm_model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf', token=hg_access_token)
        llm_model.save_pretrained(model_directory)
        print("LLama Model downloaded")

        llm_tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf', token=hg_access_token)
        llm_tokenizer.save_pretrained(tokenizer_directory)
        print("LLama Tokenizer downloaded")
        return 0

    except Exception as e:
        print(f"Failed to download Llama, error: {e}")
        return 1
    

In [ ]:
setup_llama(hg_access_token=insert_hg_key_here)

In [15]:
os.chdir("../../notebooks")

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, GenerationConfig

device = "cuda:0" if torch.cuda.is_available() else "cpu" 
llama_model = AutoModelForCausalLM.from_pretrained("../models/llama/model/", torch_dtype=torch.bfloat16, device_map="auto")
llama_tokenizer = AutoTokenizer.from_pretrained("../models/llama/tokenizer/")
print(f"Model running on {device}")

Loading checkpoint shards: 100%|██████████| 3/3 [03:18<00:00, 66.19s/it]


In [24]:
context = f"""
===========context===========
"a beach with sand dunes and blue sky",
"trumpet music to indicate a cheerful tone"
=============================

Take the following 2 contexts and create a summarised sentence with no more than 20 words and one sentence.
"""

In [28]:
context_prompt=f"""As a prompt generator for a generative AI called "Mustango", you will create music prompts for the AI to generate. I will give you a concept, and you will provide a detailed prompt for Mustango AI to generate a song. 
Please adhere to the structure and formatting below, and follow these guidelines:
•	Keep it in one sentence
•	No more than 20 words

Structure: [1] = "a beach with sand dunes and blue sky", [2]="trumpet music to indicate a cheerful tone", [3]= a sentence that takes [1] and [2] and merges them into a coherent prompt

Formatting: Follow tihs prompt structure: [3]

Example Prompts: 
Prompt 1: "Compose trumpet music for a cheerful tone on a beach with sand dunes and blue sky."
Prompt 2: "Compose a Western-inspired melody evoking the vastness of a beach with sand dunes under a blue sky."
"""

In [29]:
promptGenerator = PromptTemplate(system_prompt=context_prompt)
llama_prompt = promptGenerator.build_prompt()


In [30]:
config = GenerationConfig(max_new_tokens=1024,
                            do_sample=True,
                            top_k = 10,
                            num_return_sequences = 1,
                            return_full_text = False,
                            temperature = 0.1,
                            )
text_stream = TextStreamer(llama_tokenizer, skip_prompt=True)


encoded_input = llama_tokenizer.encode(llama_prompt, return_tensors='pt', add_special_tokens=False).to(device)
results = llama_model.generate(encoded_input, generation_config=config, streamer=text_stream)
decoded_output = llama_tokenizer.decode(results[0], skip_special_tokens=True)
response = decoded_output.split("[/INST]")[-1].strip()

t sure! I'd be happy to help generate music prompts for Mustango. Please provide the concept you'd like me to work with, and I'll create a detailed prompt for the AI to generate a song based on that concept.

Here's an example prompt based on the concepts you provided:

[3] "Compose a beach-inspired melody with a cheerful tone, featuring a trumpet solo that evokes the feeling of sand dunes and blue sky. The melody should have a lilting, carefree quality, with a sense of wide open spaces and endless horizons."

I hope this helps! Let me know if you have any other questions or if you'd like me to generate more prompts.</s>


### Mustango

In [ ]:
from sys import platform
import os

def setup_mustango(model_directory:str="../models"):
    """
    Function to download mustango model. This command makes use of os to fun terminal commands.
    Alternatively, please follow the installation process here: https://huggingface.co/declare-lab/mustango

    Args:
        model_directory (str): directory to the mustango model, defaults to models'

    Returns:
        0 for success and 1 for failure
    """

    if not os.path.isdir(model_directory):
        os.makedirs(model_directory) 
        print("Mustango Model directory not found, directory created")

    try:
        os.chdir(model_directory)
        os.system("git clone https://github.com/AMAAI-Lab/mustango")
        os.chdir("mustango")
        #edit requirements file. 
        #scikit-image, scipy has issues with installment because of version issues with matplotlib, numpy and pandas
        #torch, torchaudio and torchvision does not have cuda support in requirements file
        #solution is to remove the specific requirements
        remove_version_constraints('requirements.txt', {'torch': 'torch==2.2.1+cu121', 'torchaudio': 'torchaudio==2.2.1+cu121', 'torchvision':'torchvision==0.17.1+cu121'}, ['matplotlib', 'numpy', 'pandas', 'scikit_image', 'scipy'])
        os.system("pip install -r requirements.txt")
        os.chdir("diffusers")
        os.system("pip install -e .")
        print("Installed mustango successfully")
        return 0
    
    except Exception as e:
        print(f"Failed to download mustango, error: {e}")
        return 1


def remove_version_constraints(input_file, cuda_package, packages):
    mac_system = (platform == "darwin")
    print(f"{platform} OS detected, installing pytorch with {'cuda' if platform != 'darwin' else 'cpu'}")
    with open(input_file, 'r+') as f:
        lines = f.readlines()
        f.seek(0)
        f.truncate()
        f.write("--find-links https://download.pytorch.org/whl/torch_stable.html".rstrip('\r\n') + '\n')
        for line in lines:                
            package_name = line.split('==')[0].strip()
            if package_name in cuda_package.keys() and not mac_system:
                #not a mac sytem
        
                f.write(f"{cuda_package[package_name]}\n")
            
            elif package_name in packages:
                f.write(f"{package_name}\n")
            else:
                f.write(line)


In [ ]:
setup_mustango()


In [2]:
import os
os.chdir("../models/mustango")

In [3]:
from mustango import Mustango
model = Mustango("declare-lab/mustango")


Fetching 13 files: 100%|██████████| 13/13 [00:00<?, ?it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\valkr\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForTokenClassificationRegression were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['hidden1.bi

UNet initialized randomly.
Successfully loaded checkpoint from: declare-lab/mustango


In [4]:
import IPython
import soundfile as sf
from mustango import Mustango


prompt = "This is a new age piece. There is a flute playing the main melody with a lot of staccato notes. The rhythmic background consists of a medium tempo electronic drum beat with percussive elements all over the spectrum. There is a playful atmosphere to the piece. This piece can be used in the soundtrack of a children's TV show or an advertisement jingle."

music = model.generate(prompt)

E:\CDS Project\Landscape_Music_Generation\models\mustango\diffusers\src\diffusers\models\attention_processor.py:529: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  hidden_states = F.scaled_dot_product_attention(


In [10]:
sf.write(f"test.wav", music, samplerate=16000) #no space allowed
IPython.display.Audio(data=music, rate=16000)